In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/YLP

/content/drive/MyDrive/YLP


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torchvision.transforms as transforms
import os

In [ ]:
batch_size = 128

shift = transforms.Normalize(
    mean=[0.5,0.5,0.5],
    std=[1.,1.,1.],
  )

# ResNet takes input size 224 x 224
transformTest = transforms.Compose(
    [transforms.Resize(256),
     transforms.CenterCrop(224),
     transforms.ToTensor(),
     shift])


testSet = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transformTest)
testLoader = torch.utils.data.DataLoader(testSet, batch_size=batch_size,
                                          shuffle=False)


classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [ ]:
# Load trained network
PATH =os.path.join('results','current_resnet_20220319_0711.pt')
# net = torch.load(PATH)

# Replace output layer of ResNet to match saved size, then load weights
net = torchvision.models.resnet18(pretrained=False)
net.fc = nn.Linear(512,10)
net.load_state_dict(torch.load(PATH))
# Eval only, so dropout layers change to identity
net.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Establish baseline accuracy

net = net.to(device)

correct = 0
total = 0
with torch.no_grad():
    for data in testLoader:
        # Load data to GPU
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        # Feedforward through network
        outputs = net(images)
        # Choose predicted class as class with highest log-probability
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')
control_acc = correct / total

Accuracy of the network on the 10000 test images: 93 %


In [ ]:
print(control_acc)

0.9356


In [ ]:
hue_results = np.zeros((21,3))
row = 0

for eps in np.arange(-0.5,0.51,0.05):

  # perturb the test images by their hue
  hueTransform = transforms.Compose(
    [transforms.Resize(256),
     transforms.CenterCrop(224),
     lambda x: transforms.functional.adjust_hue(x,hue_factor=eps),
     transforms.ToTensor(),
     shift]
  )

  stabSet = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=hueTransform)
  stabLoader = torch.utils.data.DataLoader(stabSet, batch_size=batch_size,
                                            shuffle=False)
  
  correct = 0
  total = 0
# Repeat the same evaluation process as above, using perturbed images
  with torch.no_grad():
      for data in stabLoader:
          images, labels = data
          images = images.to('cuda')
          labels = labels.to('cuda')
          outputs = net(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
  acc = correct / total
  # Save results
  hue_results[row,] = np.array([eps,acc,acc/control_acc])
  row += 1

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [1]:
np.set_printoptions(suppress=True)
print(hue_results)

NameError: name 'np' is not defined